In [65]:
# from math import *
# from scipy.optimize import root
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.optimize import minimize

# %matplotlib widget
import math
from math import radians, degrees

# link lengths of front arm (mm), refer to Jack's IK sketch
a1 = 23.785 #back input link
a2 = 72.253 #shin back
a3 = 35.190 #shin top
a4 = 47.276 #shin forward
a5 = 29.621 #between shin and front input link
a6 = 25.939 #front input link
a7 = 20.200 #between input links
theta7 = radians(34.759) #angle of back input pt to front input pt to world x-axis
theta2 = radians(33.838) #angle between top and back of shin

sin7 = math.sin(theta7) 
cos7 = math.cos(theta7)

theta1_range = [radians(75.8), radians(360-6)]
theta6_range = [radians(-103.1), radians(172.3)]

neg_cos_theta_b_max = -math.cos(radians(169.297))
neg_cos_theta_t_max = -math.cos(radians(174.406))
neg_cos_theta_f_max = -math.cos(radians(142.954))

theta_b_max = radians(169.297)
theta_t_max = radians(174.406)
theta_f_max = radians(142.954)

In [26]:
import casadi
from casadi import sin, cos, atan2, pi

def fk(theta1, theta6, guess):

    opti = casadi.Opti()
    sin1 = math.sin(theta1)
    cos1 = math.cos(theta1) 
    sin6 = math.sin(theta6)
    cos6 = math.cos(theta6)

    sin3 = opti.variable()
    cos3 = opti.variable()
    sin5 = opti.variable()
    cos5 = opti.variable()

    opti.set_initial(sin3, guess[0])
    opti.set_initial(cos3, guess[1])
    opti.set_initial(sin5, guess[2])
    opti.set_initial(cos5, guess[3])

    opti.minimize((a1*cos1 + a3*cos3 - (a7*cos7 + a6*cos6 + a5*cos5))**2 + (a1*sin1 + a3*sin3 - (a7*sin7 + a6*sin6 + a5*sin5))**2)
    opti.subject_to(sin3**2 + cos3**2 == 1)
    opti.subject_to(sin5**2 + cos5**2 == 1)

    theta3 = atan2(sin3,cos3)

    p1 = (a1*cos1, a1*sin1)
    p2 = (p1[0] + a2*cos(theta3 - theta2), p1[1] + a2*sin(theta3 - theta2))
    p5 = (p1[0] + a3*cos3, p1[1] + a3*sin3)
    p7 = (a7*cos7, a7*sin7)
    p6 = (p7[0] + a6*cos6, p7[1] + a6*sin6)

    theta_b = atan2(-p1[1], -p1[0]) - atan2(p2[1] - p1[1], p2[0] - p1[0])
    # theta_b = atan2()
    theta_t = 2*pi - atan2(p7[1]-p6[1], p7[0]-p6[0]) - atan2(p5[1] - p6[1], p5[0] - p6[0])
    theta_f = atan2(p1[1]-p5[1], p1[0]-p5[0]) - atan2(p6[1] - p5[1], p6[0] - p5[0])


    opti.subject_to(opti.bounded(0, theta_b, theta_b_max))
    opti.subject_to(opti.bounded(0, theta_t, theta_t_max))
    opti.subject_to(opti.bounded(0, theta_f, theta_f_max))

    opti.solver('ipopt')


    sol = opti.solve()
    return sol, opti

theta3_guess = radians(-23.3)
theta5_guess = radians(-166.6)
sin3_g = sin(theta3_guess)
cos3_g = cos(theta3_guess)
sin5_g = sin(theta5_guess)
cos5_g = cos(theta5_guess)
guess = [sin3_g, cos3_g, sin5_g, cos5_g]

theta1_n = radians(172)
theta6_n = radians(-36)

sol, opti = fk(theta1_n, theta6_n, guess)

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        7

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:        3
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        3
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.0746763e-04 3.63e+00 8.15e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:157:
.../casadi/core/optistack_internal.cpp:1000: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Infeasible_Problem_Detected'

In [80]:
opti = casadi.Opti()
sin1 = math.sin(theta1_n)
cos1 = math.cos(theta1_n) 
sin6 = math.sin(theta6_n)
cos6 = math.cos(theta6_n)

sin3 = opti.variable()
cos3 = opti.variable()
sin5 = opti.variable()
cos5 = opti.variable()

opti.set_initial(sin3, guess[0])
opti.set_initial(cos3, guess[1])
opti.set_initial(sin5, guess[2])
opti.set_initial(cos5, guess[3])

obj = (a1*cos1 + a3*cos3 - (a7*cos7 + a6*cos6 + a5*cos5))**2 + (a1*sin1 + a3*sin3 - (a7*sin7 + a6*sin6 + a5*sin5))**2
opti.minimize(obj)
opti.subject_to(sin3**2 + cos3**2 == 1)
opti.subject_to(sin5**2 + cos5**2 == 1)

theta3 = atan2(sin3,cos3)

p1 = (a1*cos1, a1*sin1)
p2 = (p1[0] + a2*cos(theta3 - theta2), p1[1] + a2*sin(theta3 - theta2))
p5 = (p1[0] + a3*cos3, p1[1] + a3*sin3)
p7 = (a7*cos7, a7*sin7)
p6 = (p7[0] + a6*cos6, p7[1] + a6*sin6)

neg_cos_theta_b = -cos(atan2(-p1[1], -p1[0]) - atan2(p2[1] - p1[1], p2[0] - p1[0]))
neg_cos_theta_t = -cos(2*pi - atan2(p7[1]-p6[1], p7[0]-p6[0]) - atan2(p5[1] - p6[1], p5[0] - p6[0]))
neg_cos_theta_f = -cos(atan2(p1[1]-p5[1], p1[0]-p5[0]) - atan2(p6[1] - p5[1], p6[0] - p5[0]))


opti.subject_to(neg_cos_theta_b <= neg_cos_theta_b_max)
opti.subject_to(neg_cos_theta_t <= neg_cos_theta_t_max)
opti.subject_to(neg_cos_theta_f <= neg_cos_theta_f_max)
# opti.subject_to(opti.bounded(0, theta_t, theta_t_max))
# opti.subject_to(opti.bounded(0, theta_f, theta_f_max))

p_opts = dict(print_time=False, verbose=False)
s_opts = dict(print_level=0)
# opti.solver('ipopt', p_opts, s_opts)
opti.solver('ipopt')
sol = opti.solve()


sol.value(p2[0]), sol.value(p2[1]), sol.value(obj)


This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        7

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:        3
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        3

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.0746763e-04 3.34e-03 8.15e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

(15.478892350740729, -57.492451332472264, 0.003965973340652619)

In [88]:
def fk(theta1, theta6, guess=None):
    if(guess == None):
        theta3_guess = radians(-23.3)
        theta5_guess = radians(-166.6)
        sin3_g = sin(theta3_guess)
        cos3_g = cos(theta3_guess)
        sin5_g = sin(theta5_guess)
        cos5_g = cos(theta5_guess)
        guess = [sin3_g, cos3_g, sin5_g, cos5_g]
        
    opti = casadi.Opti()
    sin1 = math.sin(theta1)
    cos1 = math.cos(theta1) 
    sin6 = math.sin(theta6)
    cos6 = math.cos(theta6)

    sin3 = opti.variable()
    cos3 = opti.variable()
    sin5 = opti.variable()
    cos5 = opti.variable()

    opti.set_initial(sin3, guess[0])
    opti.set_initial(cos3, guess[1])
    opti.set_initial(sin5, guess[2])
    opti.set_initial(cos5, guess[3])

    obj = (a1*cos1 + a3*cos3 - (a7*cos7 + a6*cos6 + a5*cos5))**2 + (a1*sin1 + a3*sin3 - (a7*sin7 + a6*sin6 + a5*sin5))**2
    opti.minimize(obj)
    opti.subject_to(sin3**2 + cos3**2 == 1)
    opti.subject_to(sin5**2 + cos5**2 == 1)

    theta3 = atan2(sin3,cos3)

    p1 = (a1*cos1, a1*sin1)
    p2 = (p1[0] + a2*cos(theta3 - theta2), p1[1] + a2*sin(theta3 - theta2))
    p5 = (p1[0] + a3*cos3, p1[1] + a3*sin3)
    p7 = (a7*cos7, a7*sin7)
    p6 = (p7[0] + a6*cos6, p7[1] + a6*sin6)

    neg_cos_theta_b = -cos(atan2(-p1[1], -p1[0]) - atan2(p2[1] - p1[1], p2[0] - p1[0]))
    neg_cos_theta_t = -cos(2*pi - atan2(p7[1]-p6[1], p7[0]-p6[0]) - atan2(p5[1] - p6[1], p5[0] - p6[0]))
    neg_cos_theta_f = -cos(atan2(p1[1]-p5[1], p1[0]-p5[0]) - atan2(p6[1] - p5[1], p6[0] - p5[0]))

    opti.subject_to(neg_cos_theta_b <= neg_cos_theta_b_max)
    opti.subject_to(neg_cos_theta_t <= neg_cos_theta_t_max)
    opti.subject_to(neg_cos_theta_f <= neg_cos_theta_f_max)

    p_opts = dict(print_time=False, verbose=False)
    s_opts = dict(print_level=0)
    opti.solver('ipopt', p_opts, s_opts)
    # opti.solver('ipopt')
    sol = opti.solve()

    return sol.value(p2[0]), sol.value(p2[1]), sol.value(obj)

theta1_n = radians(172)
theta6_n = radians(-36)

fk(theta1_n, theta6_n)

(15.478892350740729, -57.492451332472264, 0.003965973340652619)